<a href="https://colab.research.google.com/github/pedroteche-ih/79_PT_JUN2022/blob/main/79%20DA%20PT%20JUN-2022%20Deep%20Learning%20com%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split

# Deep Learning

## Modelos de Classificação

In [2]:
url = 'https://raw.githubusercontent.com/pedroteche-ih/64_PT_NOV202111/main/aulas/data/tb_hotel_completa.csv'
tb_hotel = pd.read_csv(url)
tb_hotel['is_company'] = np.where(tb_hotel['company'].isna(), 0, 1)
tb_hotel['is_agent'] = np.where(tb_hotel['agent'].isna(), 0, 1)

In [3]:
tb_hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 33 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      119390 non-null  int64  
 1   hotel                           119390 non-null  object 
 2   is_canceled                     119390 non-null  int64  
 3   lead_time                       119390 non-null  int64  
 4   stays_in_weekend_nights         119390 non-null  int64  
 5   stays_in_week_nights            119390 non-null  int64  
 6   adults                          119390 non-null  int64  
 7   children                        119386 non-null  float64
 8   babies                          119390 non-null  int64  
 9   meal                            119390 non-null  object 
 10  country                         118902 non-null  object 
 11  market_segment                  119390 non-null  object 
 12  distribution_cha

## Construindo Pipelines

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [5]:
tb_hotel.select_dtypes(include = "number").columns

Index(['Unnamed: 0', 'is_canceled', 'lead_time', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests', 'id_booking', 'is_company', 'is_agent'],
      dtype='object')

In [6]:
cat_vars = [
    'hotel', 'meal', 'country', 
    'market_segment', 'distribution_channel',
    'reserved_room_type', 'assigned_room_type', 
    'deposit_type', 'customer_type', 'is_company', 'is_agent'
]
num_vars = [
    'lead_time', 'stays_in_weekend_nights',
    'stays_in_week_nights', 'adults', 'children', 'babies',
    'is_repeated_guest', 'previous_cancellations',
    'previous_bookings_not_canceled','days_in_waiting_list', 
    'adr', 'required_car_parking_spaces',
    'total_of_special_requests'
]

#### Braço Numérico

In [7]:
num_imputer = KNNImputer(n_neighbors = 5, weights = 'distance')
num_scaler = StandardScaler()
num_pipeline = Pipeline([('IMPUTER', num_imputer), ('SCALER', num_scaler)])


#### Braço Categórico

In [8]:
cat_imputer = SimpleImputer(strategy = 'constant', fill_value = 'Unknown')
ohe = OneHotEncoder(
    drop = 'first', 
    handle_unknown = 'ignore',
    min_frequency = 10
)
cat_pipeline = Pipeline([('IMPUTER', cat_imputer), ('OHE', ohe)])

#### Pipeline Final

In [9]:
composed = ColumnTransformer([('CAT', cat_pipeline, cat_vars),
                              ('NUM', num_pipeline, num_vars)])
dataprep_pipeline = Pipeline([('DATAPREP', composed)])

In [10]:
X = tb_hotel[cat_vars + num_vars]
y = np.array(tb_hotel['is_canceled'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [11]:
dataprep_pipeline.fit(X_train)
X_train_trans = dataprep_pipeline.transform(X_train)
X_test_trans = dataprep_pipeline.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [12]:
num_features = X_train_trans.shape[1]
print(num_features)

146


## Criando nossa Rede Neural

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [14]:
%load_ext tensorboard

#### Definindo a topologia

Na definição da topologia precisamos prestar atenção à alguns parametros:

1. `input_dim` na primeira camada deve ser o número de features em nosso modelo;
1. `activation` nas camadas escondidas é hiperparâmetro a ser testado;
1. `activation` na última camada é uma função do tipo de modelo que queremos construir:
  * "sigmoid" para classificação binária;
  * "softmax" para multi-classificação;
  * não precisa ser especificada para problemas de regressão;
1. o **tamanho da última camada** deve corresponder ao tipo de previsão que queremos fazer:
  * **1** para problemas de classificação binária;
  * **n** para problemas de classificação com *n* categorias;
  * **1** para problemas de regressão.

Além das camadas densas (como as camadas de um MLP) vamos adicionar uma camada de `Dropout`: 

1. Camadas de *dropout* tem uma probabilidade de não passar as informações de alguns neurônios de uma camada para a próxima - essa técnica ajuda redes profundas a evitar overfitting.
1. O único hiperparâmetro de uma cada `Dropout` é *p*, a % de neurônios que são bloqueados em cada batch.

In [15]:
model = Sequential()
model.add(Dense(30, input_dim=num_features, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

Agora podemos compilar nosso modelo. O parâmetro fundamental dessa função é a *loss function*, que é consequência do problema que queremos resolver:

1. `binary_crossentropy` para problemas de classificação binária;
1. `categorical_crossentropy` para problemas de multi-classificação;
1. `mean_squared_error` para problemas de regressão.

https://keras.io/api/losses/

De forma semelhante, a métrica utilizada precisa refletir o tipo de problema que estamos resolvendo:

https://keras.io/api/metrics/

In [16]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

#### Estimando pesos

O principal hiperparâmetro em um modelo de Deep Learning é o número de *épocas* (`epochs`) - que representa por quantas rodadas de otimização nossa rede passará. Um número maior de épocas melhora nosso modelo mas pode causar overfitting!

In [17]:
h = model.fit(X_train_trans.toarray(), y_train, epochs = 25, batch_size = 64, validation_split = 0.1)

Epoch 1/25
1344/1344 [==============================] - 4s 2ms/step - loss: 0.4228 - accuracy: 0.7988 - val_loss: 0.3463 - val_accuracy: 0.8372
Epoch 2/25
1344/1344 [==============================] - 2s 2ms/step - loss: 0.3606 - accuracy: 0.8299 - val_loss: 0.3365 - val_accuracy: 0.8424
Epoch 3/25
1344/1344 [==============================] - 2s 2ms/step - loss: 0.3489 - accuracy: 0.8364 - val_loss: 0.3253 - val_accuracy: 0.8459
Epoch 4/25
1344/1344 [==============================] - 2s 2ms/step - loss: 0.3410 - accuracy: 0.8397 - val_loss: 0.3220 - val_accuracy: 0.8468
Epoch 5/25
1344/1344 [==============================] - 2s 2ms/step - loss: 0.3377 - accuracy: 0.8403 - val_loss: 0.3187 - val_accuracy: 0.8500
Epoch 6/25
1344/1344 [==============================] - 2s 2ms/step - loss: 0.3332 - accuracy: 0.8445 - val_loss: 0.3167 - val_accuracy: 0.8510
Epoch 7/25
1344/1344 [==============================] - 2s 2ms/step - loss: 0.3307 - accuracy: 0.8456 - val_loss: 0.3126 - val_accuracy:

In [18]:
y_pred_prob = model.predict(X_test_trans)
y_pred_prob

747/747 [==============================] - 1s 936us/step


array([[1.5548855e-16],
       [1.0000000e+00],
       [3.9843023e-02],
       ...,
       [3.7726966e-01],
       [6.2349039e-01],
       [1.0000000e+00]], dtype=float32)

In [28]:
y_pred = [1 if x > 0.5 else 0 for x in y_pred_prob]
y_pred[0:10]

[0, 1, 0, 0, 0, 0, 0, 0, 0, 1]

In [20]:
from sklearn.metrics import f1_score
print(f"F1-Scoreo no Conjunto de Teste: {np.round(f1_score(y_test, y_pred), 3)}")

F1-Scoreo no Conjunto de Teste: 0.793


### Visualizando Overfitting

Para visualizar overfitting, vamos criar uma rede profunda, com mais camadas:

In [29]:
model_2 = Sequential()
model_2.add(Dense(80, input_dim=num_features, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(1, activation="sigmoid"))
model_2.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

In [30]:
history = model_2.fit(
    X_train_trans.toarray(), y_train,
    validation_split = 0.1, 
    epochs=25, batch_size=64)

Epoch 1/25
1344/1344 [==============================] - 6s 3ms/step - loss: 0.3837 - precision_1: 0.8066 - recall_1: 0.6532 - val_loss: 0.3357 - val_precision_1: 0.8345 - val_recall_1: 0.7156
Epoch 2/25
1344/1344 [==============================] - 4s 3ms/step - loss: 0.3407 - precision_1: 0.8164 - recall_1: 0.7258 - val_loss: 0.3287 - val_precision_1: 0.8308 - val_recall_1: 0.7379
Epoch 3/25
1344/1344 [==============================] - 4s 3ms/step - loss: 0.3292 - precision_1: 0.8249 - recall_1: 0.7313 - val_loss: 0.3188 - val_precision_1: 0.8249 - val_recall_1: 0.7612
Epoch 4/25
1344/1344 [==============================] - 4s 3ms/step - loss: 0.3236 - precision_1: 0.8308 - recall_1: 0.7330 - val_loss: 0.3139 - val_precision_1: 0.8259 - val_recall_1: 0.7603
Epoch 5/25
1344/1344 [==============================] - 4s 3ms/step - loss: 0.3185 - precision_1: 0.8236 - recall_1: 0.7568 - val_loss: 0.3097 - val_precision_1: 0.8006 - val_recall_1: 0.8048
Epoch 6/25
1344/1344 [==================

In [31]:
history.history.keys()

dict_keys(['loss', 'precision_1', 'recall_1', 'val_loss', 'val_precision_1', 'val_recall_1'])

In [32]:
tb_eval = pd.DataFrame({'precision' : history.history['precision'], 
                        'recall' : history.history['recall'], 
                        'val_precision' : history.history['val_precision'], 
                        'val_recall' : history.history['val_recall'], 
                        'epoch' : range(25)})

KeyError: ignored

In [ ]:
tb_eval['f1_score'] = 2 * (tb_eval['precision'] * tb_eval['recall'])/(tb_eval['precision'] + tb_eval['recall'])
tb_eval['val_f1_score'] = 2 * (tb_eval['val_precision'] * tb_eval['val_recall'])/(tb_eval['val_precision'] + tb_eval['val_recall'])

In [ ]:
sns.lineplot(data = tb_eval, x = 'epoch', y = 'f1_score', label = "Train")
sns.lineplot(data = tb_eval, x = 'epoch', y = 'val_f1_score', label = "Test")

In [33]:
y_pred_prob = model_2.predict(X_test_trans)
y_pred = [1 if x > 0.5 else 0 for x in y_pred_prob]
print(f"F1-Score no Conjunto de Teste: {np.round(f1_score(y_test, y_pred), 3)}")

747/747 [==============================] - 1s 1ms/step
F1-Score no Conjunto de Teste: 0.81


In [27]:
# 